In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import torch
import pdb
from sklearn.metrics import roc_auc_score
np.random.seed(2020)
torch.manual_seed(2020)
import pdb

from dataset import load_data
from matrix_factorization import MF, MF_VITA, MF_IPS, MF_SNIPS
from utils import gini_index, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)

dataset_name = "yahoo"

In [2]:
if dataset_name == "coat":
    train_mat, test_mat = load_data("coat")        
    x_train, y_train = rating_mat_to_sample(train_mat)
    x_test, y_test = rating_mat_to_sample(test_mat)
    num_user = train_mat.shape[0]
    num_item = train_mat.shape[1]

elif dataset_name == "yahoo":
    x_train, y_train, x_test, y_test = load_data("yahoo")
    x_train, y_train = shuffle(x_train, y_train)
    num_user = x_train[:,0].max() + 1
    num_item = x_train[:,1].max() + 1

print("# user: {}, # item: {}".format(num_user, num_item))
# binarize
y_train = binarize(y_train)
y_test = binarize(y_test)

===>Load from yahoo data set<===
[train] num data: 311704
[test]  num data: 54000
# user: 15401, # item: 1001


In [3]:
"MF VITA"
mf_vita = MF_VITA(num_user, num_item)
mf_vita.fit(x_train, y_train, 
    lr=0.01,
    batch_size=2048,
    lamb=1e-5,
    alpha=0.1,
    tol=1e-6,
    verbose=False)

test_pred = mf_vita.predict(x_test)
mse_mf = mse_func(y_test, test_pred)
auc_mf = roc_auc_score(y_test, test_pred)
print("***"*5 + "[MF-VITA]" + "***"*5)
print("[MF-VITA] test mse:", mse_func(y_test, test_pred))
print("[MF-VITA] test auc:", auc_mf)
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF-VITA]" + "***"*5)

[MF-VITA] epoch:26, xent:64.95301312208176
***************[MF-VITA]***************
[MF-VITA] test mse: 0.16710671711830904
[MF-VITA] test auc: 0.7163256119159136
Num User: 5400
Gini index: 0.5272308963238461
Global utility: 0.29888888888888887
***************[MF-VITA]***************


In [4]:
"MF naive"
mf = MF(num_user, num_item)
mf.fit(x_train, y_train, lr=0.01, batch_size=2048, lamb=1e-4,
    verbose=0)
test_pred = mf.predict(x_test)
mse_mf = mse_func(y_test, test_pred)
auc_mf = roc_auc_score(y_test, test_pred)
print("***"*5 + "[MF]" + "***"*5)
print("[MF] test mse:", mse_func(y_test, test_pred))
print("[MF] test auc:", auc_mf)
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF]" + "***"*5)

[MF] epoch:17, xent:97.3810361623764
***************[MF]***************
[MF] test mse: 0.2493406519314255
[MF] test auc: 0.6767433527074862
Num User: 5400
Gini index: 0.5853821558742106
Global utility: 0.2514074074074074
***************[MF]***************


In [5]:
"MF IPS"
mf_ips = MF_IPS(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_ips.fit(x_train, y_train, y_ips=y_ips,
    lr=0.05, 
    batch_size=2048,
    lamb=1e-4,verbose=0)
test_pred = mf_ips.predict(x_test)
mse_mfips = mse_func(y_test, test_pred)
auc_mfips = roc_auc_score(y_test, test_pred)
print("***"*5 + "[MF-IPS]" + "***"*5)
print("[MF-IPS] test mse:", mse_func(y_test, test_pred))
print("[MF-IPS] test auc:", auc_mfips)
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF-IPS]" + "***"*5)

[MF-IPS] epoch:17, xent:2520.2817039489746
***************[MF-IPS]***************
[MF-IPS] test mse: 0.2259804385142548
[MF-IPS] test auc: 0.6792506681279089
Num User: 5400
Gini index: 0.566583361558695
Global utility: 0.2551481481481482
***************[MF-IPS]***************


In [6]:
"MF-SNIPS"
mf_snips = MF_SNIPS(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

mf_snips.fit(x_train, y_train, y_ips=y_ips,lr=0.05,
        batch_size=2048, lamb=1e-5,
        verbose=0)

test_pred = mf_snips.predict(x_test)
mse_mfsnips = mse_func(y_test, test_pred)
auc_mfsnips = roc_auc_score(y_test, test_pred)
print("***"*5 + "[MF-SNIPS]" + "***"*5)
print("[MF-SNIPS] test mse:", mse_mfsnips)
print("[MF-SNIPS] test auc:", auc_mfsnips)
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[MF-SNIPS]" + "***"*5)

[MF-SNIPS] epoch:12, xent:58.32837986946106
***************[MF-SNIPS]***************
[MF-SNIPS] test mse: 0.19450282408800773
[MF-SNIPS] test auc: 0.6810477398623035
Num User: 5400
Gini index: 0.5651779289270158
Global utility: 0.25048148148148147
***************[MF-SNIPS]***************
